设置仿真参数

In [95]:
import random
import simpy

RANDOM_SEED = 42
REQ_TOTAL = 100 # 总请求数
ARRIVAL_RATE =   0.05 # 请求到达率 (每毫秒来多少个请求)
SERVICE_RATE =   0.1  # 服务速率   (每毫秒做多少个请求)
OVER_TIME    = 100.0  # 请求超时   (毫秒)

latency = [] # 采集请求延迟


初始化仿真对象

In [96]:
def workload(env, number, arr_rate, server):
    """基于所设均值泊松分布生成请求"""
    for i in range(number):
        c = req_handler(env, 'Request%03d' % i, server, svr_rate=SERVICE_RATE)
        env.process(c)
        t = random.expovariate(arr_rate)
        yield env.timeout(t)

def req_handler(env, name, server, svr_rate):
    """请求处理过程"""
    arrive = env.now
    print('%7.4f %s: Incoming request' % (arrive, name))

    with server.request() as req:
        # ot = random.uniform(MIN_PATIENCE, MAX_PATIENCE)
        ot = random.uniform(1, 3)
        # 等待服务或超时
        results = yield req | env.timeout(OVER_TIME)

        wait = env.now - arrive

        if req in results:
            # We got to the server
            print('%7.4f %s: Waited %6.3f' % (env.now, name, wait))

            tib = random.expovariate(svr_rate)
            yield env.timeout(tib)
            print('%7.4f %s: Finished' % (env.now, name))

            # 请求正常完成，采集延迟
            latency.append(env.now - arrive)

        else:
            # 请求超时
            print('%7.4f %s: Overtime after %6.3f' % (env.now, name, wait))

# 仿真环境准备
random.seed(RANDOM_SEED)
env = simpy.Environment()

开始仿真

In [97]:
server = simpy.Resource(env, capacity=1)
env.process(workload(env, REQ_TOTAL, ARRIVAL_RATE, server))
env.run()

 0.0000 Request000: Incoming request
 0.0000 Request000: Waited  0.000
 3.2162 Request000: Finished
20.4012 Request001: Incoming request
20.4012 Request001: Waited  0.000
25.4529 Request002: Incoming request
31.6929 Request001: Finished
31.6929 Request002: Waited  6.240
37.1734 Request002: Finished
69.9987 Request003: Incoming request
69.9987 Request003: Waited  0.000
70.6037 Request004: Incoming request
71.1416 Request005: Incoming request
77.0378 Request003: Finished
77.0378 Request004: Waited  6.434
79.5281 Request004: Finished
79.5281 Request005: Waited  8.387
88.4262 Request005: Finished
92.1314 Request006: Incoming request
92.1314 Request006: Waited  0.000
108.5211 Request006: Finished
125.2862 Request007: Incoming request
125.2862 Request007: Waited  0.000
126.9760 Request007: Finished
149.2420 Request008: Incoming request
149.2420 Request008: Waited  0.000
150.2153 Request008: Finished
212.2724 Request009: Incoming request
212.2724 Request009: Waited  0.000
214.3068 Request010:

记录延迟数据

In [98]:
with open("latency.csv", "w+") as tracefile:
    tracefile.write("# latency\n")
    tracefile.writelines([str(l) + '\n' for l in latency])